# Notebook #2: Limpieza: limpieza de datos, organización de columnas y formato

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../src')
import funciones as f
import soporte_prep as sp

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

In [2]:
df_principal = pd.read_csv("../../Datos/datos_defunciones/BD_MUERTES_ACCIDENTES_2007-2023.csv", sep='\t')

In [3]:
# Lista de nombres de archivos
filenames = [
    'df_diasem.csv',
    'df_mes.csv',
    'df_rhora.csv',
    'df_region.csv',
    'df_provincia.csv',
    'df_tipovia.csv',
    'df_accidente.csv',
    'df_mediot.csv',
    'df_genero.csv',
    'df_gedad.csv',
    'df_victima.csv'
]

# Crear un diccionario para almacenar los DataFrames leídos
df_dict = {}

# Leer cada archivo y almacenarlo en el diccionario
for filename in filenames:
    df_name = filename.split('.')[0]  # Usar el nombre del archivo (sin extensión) como clave
    df_dict[df_name] = pd.read_csv(f'../../Datos/datos_defunciones/{filename}', index_col=0)

In [4]:
df_principal.drop(columns='CAUSA_M', inplace=True)

In [5]:
df_principal['FECHA'] = pd.to_datetime(df_principal['ANOOCU'].astype(str) + '-' + 
                                    df_principal['MESOCU'].astype(str) + '-' + 
                                    '28', 
                                    format='%Y-%m-%d')
# creo la columna de fecha completa
df_principal.head()

,REGION,PROVINCIA,DIASEMANA,MESOCU,ANOOCU,RHORA,TVIA,SEXO,GRUPOEDAD,CONDIVICTIMA,TIPOACCIDENTE,MEDIOTRANS,ESTRECON,VEHICHOQUE,VEHICUVOLC,VEHICUATRO,VEHICAIDA,VEHIDESLI,APLASTAM,OTROACCIDEN,FECHA
0,1,9,6,1,2007,10,99,1,5,9,1,2,,8,,,,,,,2007-01-28
1,1,9,7,1,2007,1,4,1,12,9,1,2,,1,,,,,,,2007-01-28
2,1,9,4,2,2007,11,4,1,4,9,1,2,,1,,,,,,,2007-02-28
3,1,9,1,2,2007,6,99,1,1,9,3,25,,,,3,,,,,2007-02-28
4,1,9,4,3,2007,10,4,1,5,9,1,2,,4,,,,,,,2007-03-28


In [6]:
df_llena = df_principal.copy()

In [7]:
ultima_columna = df_llena.columns[-1] 
columnas = [ultima_columna] + [col for col in df_llena.columns if col != ultima_columna]
df_llena = df_llena[columnas]

In [8]:
df_llena = df_llena.iloc[:,0:13] #corto las columnas que quiero

In [9]:
df_codificada = df_llena.copy()

In [10]:
# lista de tuplas con las columnas a mapear, el df correspondiente y la columna única
mapeos = [
    ('TIPOACCIDENTE', df_dict['df_accidente']), ('REGION', df_dict['df_region']), ('PROVINCIA', df_dict['df_provincia']), ('DIASEMANA', df_dict['df_diasem']), ('MESOCU', df_dict['df_mes']), ('RHORA', df_dict['df_rhora']),
    ('TVIA', df_dict['df_tipovia']), ('SEXO', df_dict['df_genero']),('GRUPOEDAD', df_dict['df_gedad']), ('CONDIVICTIMA', df_dict['df_victima']), ('MEDIOTRANS', df_dict['df_mediot']),
]

for columna, df_mapeado in mapeos:
    mapeo = dict(zip(df_mapeado['Valor'], df_mapeado['Significado']))
    valores = df_llena[columna].unique()
    mapeo_filtrado = {k: v for k, v in mapeo.items() if k in valores}
    df_llena[columna] = df_llena[columna].map(mapeo_filtrado)

In [11]:
df_llena.columns = ['fecha', 'region', 'provincia', 'dia_semana', 'mes', 'anio', 'rango_hora', 'tipo_via', 'sexo', 'grupo_edad', 'condicion_victima', 'tipo_accidente', 'medio_trans']
df_codificada.columns = ['fecha', 'region', 'provincia', 'dia_semana', 'mes', 'anio', 'rango_hora', 'tipo_via', 'sexo', 'grupo_edad', 'condicion_victima', 'tipo_accidente', 'medio_trans']

In [12]:
df_descodificada = df_llena.copy()

In [13]:
df_descodificada.to_csv('../../Datos/datos_defunciones/df_descodificada.csv')
df_codificada.to_csv('../../Datos/datos_defunciones/df_codificada.csv')

In [14]:
df_descodificada.head()

,fecha,region,provincia,dia_semana,mes,anio,rango_hora,tipo_via,sexo,grupo_edad,condicion_victima,tipo_accidente,medio_trans
0,2007-01-28,Región Cibao Norte,Espaillat,Sábado,Enero,2007,18:01-20:00,No declarada,Hombres,20-24,No declarada,Choque o colisión,"Motocicleta, ciclomotor (passola)"
1,2007-01-28,Región Cibao Norte,Espaillat,Domingo,Enero,2007,00:01-02:00,Carretera,Hombres,55-59,No declarada,Choque o colisión,"Motocicleta, ciclomotor (passola)"
2,2007-02-28,Región Cibao Norte,Espaillat,Jueves,Febrero,2007,20:01-22:00,Carretera,Hombres,15-19,No declarada,Choque o colisión,"Motocicleta, ciclomotor (passola)"
3,2007-02-28,Región Cibao Norte,Espaillat,Lunes,Febrero,2007,10:01-12:00,No declarada,Hombres,0-4,No declarada,Atropellamiento,Peatón atropellado
4,2007-03-28,Región Cibao Norte,Espaillat,Jueves,Marzo,2007,18:01-20:00,Carretera,Hombres,20-24,No declarada,Choque o colisión,"Motocicleta, ciclomotor (passola)"


In [ ]:
df_descodificada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31351 entries, 0 to 31350
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha              31351 non-null  datetime64[ns]
 1   region             31351 non-null  object        
 2   provincia          31351 non-null  object        
 3   dia_semana         31351 non-null  object        
 4   mes                31351 non-null  object        
 5   anio               31351 non-null  int64         
 6   rango_hora         31351 non-null  object        
 7   tipo_via           31351 non-null  object        
 8   sexo               31351 non-null  object        
 9   grupo_edad         31351 non-null  object        
 10  condicion_victima  31351 non-null  object        
 11  tipo_accidente     31351 non-null  object        
 12  medio_trans        31351 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 3.1+